# Imports

In [ ]:
import nbimporter
import Functions
import numpy as np
import pandas as pd
import re

In [174]:
data = Functions.load_data('cleandata.csv')
data.head()

,line_text,speaker
0,All right Jim. Your quarterlies look very good...,michael
1,"Oh, I told you. I couldn't close it. So...",jim
2,So you've come to the master for guidance? Is ...,michael
3,"Actually, you called me in here, but yeah.",jim
4,"All right. Well, let me show you how it's done.",michael


# Convert sentences to GloVe

In [175]:
# MOVE to functions
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

#https://stackoverflow.com/questions/37793118/load-pretrained-glove-vectors-in-python   

In [176]:
glove = loadGloveModel('glove.6B.50d.txt')

Loading Glove Model
Done. 400000  words loaded!


In [177]:
glovesentences = []
text = data['line_text']
for line in text:
    glovewords = []
    line = line.lower()
    for word in line.split():
        try:
            rep = glove[word]
        except:
            rep = [0] * 50
        glovewords.append(rep)
    glovesentences.append(glovewords)

In [189]:
data['glove'] = glovesentences
data.head()

,line_text,speaker,glove
0,all right jim . your quarterlies look very goo...,michael,"[[0.19253, 0.10006, 0.063798, -0.087664, 0.522..."
1,"oh , i told you . i couldn't close it . so . . .",jim,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,so you've come to the master for guidance ? is...,michael,"[[0.60308, -0.32024, 0.088857, -0.55176, 0.531..."
3,"actually , you called me in here , but yeah .",jim,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"all right . well , let me show you how it's do...",michael,"[[0.19253, 0.10006, 0.063798, -0.087664, 0.522..."


In [188]:
# MOVE TO CLEAN
data['line_text'] = data['line_text'].str.replace("[\(\[].*?[\)\]]", "")
data['line_text'] = data['line_text'].str.replace('([.,!?()])', r' \1 ')
data['line_text'] = data['line_text'].str.replace('\s{2,}', ' ')
data['line_text'] = data['line_text'].str.lower()
data.head()

,line_text,speaker,glove
0,all right jim . your quarterlies look very goo...,michael,"[[0.19253, 0.10006, 0.063798, -0.087664, 0.522..."
1,"oh , i told you . i couldn't close it . so . . .",jim,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,so you've come to the master for guidance ? is...,michael,"[[0.60308, -0.32024, 0.088857, -0.55176, 0.531..."
3,"actually , you called me in here , but yeah .",jim,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,"all right . well , let me show you how it's do...",michael,"[[0.19253, 0.10006, 0.063798, -0.087664, 0.522..."


# Feed GloVe representations to Transformer